# Create semantic segmentation dataset

This notebook shows how to create a semantic segmentation table from a dataset of images and segmentation masks (grayscale PNG images).

## Imports

In [ ]:
import json
from pathlib import Path

import tlc
from huggingface_hub import hf_hub_download

from tlc_tools.common import download_and_extract_zipfile

## Project Setup

In [ ]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "ADE20k_toy_dataset"
TABLE_NAME = "ADE20K-semantic-segmentation"
TRANSIENT_DATA_PATH = "../../transient_data"

## Download the dataset

In [ ]:
DATASET_ROOT = (Path(TRANSIENT_DATA_PATH) / "ADE20k_toy_dataset").resolve()

if not DATASET_ROOT.exists():
    print("Downloading data...")
    download_and_extract_zipfile(
        url="https://www.dropbox.com/s/l1e45oht447053f/ADE20k_toy_dataset.zip?dl=1",
        location=TRANSIENT_DATA_PATH,
    )

## Fetch the label map from the Hugging Face Hub

In [ ]:
# load id2label mapping from a JSON on the hub
with open(
    hf_hub_download(
        repo_id="huggingface/label-files",
        filename="ade20k-id2label.json",
        repo_type="dataset",
    )
) as f:
    id2label = json.load(f)

categories = list(id2label.values())

In [ ]:
value_map = {i + 1: tlc.MapElement(category) for i, category in enumerate(categories)}
value_map[0] = tlc.MapElement("background", display_color="#00000000")  # Set transparent background

In [ ]:
tlc.register_project_url_alias("ADE20K_TOY_DATASET_ROOT", DATASET_ROOT.as_posix(), project=PROJECT_NAME)

## Load the images and segmentation maps





In [ ]:
image_paths = list(DATASET_ROOT.glob("**/images/training/*.jpg"))
segmentation_map_paths = list(DATASET_ROOT.glob("**/annotations/training/*.png"))

In [ ]:
# Call .to_relative() to ensure aliases are applied
image_paths = [tlc.Url(p).to_relative().to_str() for p in image_paths]
mask_paths = [tlc.Url(p).to_relative().to_str() for p in segmentation_map_paths]
print(image_paths[0])

## Write the instance segmentation masks to a table

In [ ]:
table = tlc.Table.from_dict(
    data={
        "image": image_paths,
        "mask": mask_paths,
    },
    structure=(tlc.PILImage("image"), tlc.SegmentationPILImage("mask", classes=value_map)),
    table_name=TABLE_NAME,
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
)

In [ ]:
image, mask = table[0]

In [ ]:
image

In [ ]:
mask